# **Building a simple RAG**

In [55]:
import langchain
langchain.__version__

'1.0.5'

In [66]:
# importing necessary libraries
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
import os

# loading the environment variables
load_dotenv()



True

In [58]:
# Loading the data
loader = TextLoader("../data/AI.txt", encoding='utf-8')
documents = loader.load()

In [59]:
documents

[Document(metadata={'source': '../data/AI.txt'}, page_content='The Evolution and Impact of Artificial Intelligence: From Theory to Practice\n\nArtificial Intelligence (AI) has transformed from a theoretical concept in the 1950s into one of the most influential technologies of the 21st century. The field encompasses multiple disciplines including machine learning, natural language processing, computer vision, and robotics, with applications spanning healthcare, finance, education, and beyond.\nHistorical Foundations and Key Milestones\n\nThe term "Artificial Intelligence" was coined by John McCarthy in 1956 at the Dartmouth Conference, marking the official birth of AI as an academic discipline. Early pioneers like Alan Turing proposed the famous Turing Test in 1950, asking whether machines could exhibit intelligent behavior indistinguishable from humans. The first AI programs emerged in the late 1950s, including the Logic Theorist (1956) and the General Problem Solver (1957).\n\nThe fie

In [60]:
# splitting the data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [25]:
chunks[0]

Document(metadata={'source': '../data/AI.txt'}, page_content='The Evolution and Impact of Artificial Intelligence: From Theory to Practice\n\nArtificial Intelligence (AI) has transformed from a theoretical concept in the 1950s into one of the most influential technologies of the 21st century. The field encompasses multiple disciplines including machine learning, natural language processing, computer vision, and robotics, with applications spanning healthcare, finance, education, and beyond.\nHistorical Foundations and Key Milestones')

In [43]:
# embedding the chunks
embeddings = OpenAIEmbeddings()

In [ ]:
# creating the vector store
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
# fetching the most relevant documents to query from vector store
query = "How many parameters does GPT-3 have?"
docs = vectorstore.similarity_search(query, k=4)

for i, doc in enumerate(docs):
    print(f"Document {i+1}:")
    print(doc.page_content)
    print("*"*50)

Document 1:
Large Language Models (LLMs) like GPT-4, Claude, and PaLM demonstrate remarkable capabilities across diverse tasks. These models undergo two-stage training: pre-training on massive text corpora (often 1+ trillion tokens) to learn language patterns, followed by fine-tuning on specific tasks or alignment with human preferences through Reinforcement Learning from Human Feedback (RLHF).
**************************************************
Document 2:
The transformer architecture, introduced in the seminal 2017 paper "Attention Is All You Need" by Vaswani et al., revolutionized NLP. Transformers use self-attention mechanisms to process entire sequences simultaneously, enabling parallel computation and capturing long-range dependencies more effectively than recurrent neural networks (RNNs). This architecture spawned groundbreaking models like BERT (Bidirectional Encoder Representations from Transformers) with 340 million parameters and GPT-3 with 175 billion parameters.
***********

In [62]:
# creating the chat model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [67]:
# defining the prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [68]:
# defining prompt
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [71]:
# defining output parser
str_parser = StrOutputParser()

In [72]:
# defining chain
chain = {'context': retriever, 'question': RunnablePassthrough()}| prompt| llm | str_parser

In [73]:
# getting inference
final_answer = chain.invoke("How many parameters does GPT-3 have?")
print(final_answer)

c:\Users\masan\OneDrive\Desktop\Advanced End to End RAG Application\.venv\Lib\site-packages\pydantic\v1\main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)


GPT-3 has 175 billion parameters. This large number of parameters contributes to its ability to understand and generate human-like text across various tasks. The model's architecture is based on the transformer framework, which allows it to process sequences of text effectively. This capability is a significant advancement over previous models, enabling better performance in natural language processing tasks.
